In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 99)

In [2]:
airports = pd.read_csv('airports.csv')

In [3]:
planes = pd.read_csv('planes.csv')

In [4]:
#flights = pd.read_csv('flights_test_size.csv')
flights = pd.read_csv('flights.csv')
print('csv read complete')
#create frame of all dates with flights, will automatically account for leap-years since it's based on real data
byDate = pd.DataFrame((flights['MONTH'].astype(str) + '/' + flights['DAY'].astype(str) + '/' + flights['YEAR'].astype(str)).unique())
byDate.columns = ['DATE']

C:\Users\frype\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


csv read complete


In [5]:
#concatinate the dates of flights to a single column
flights=flights[flights['CANCELLED']==0]
flights['DATE'] = flights['MONTH'].astype(str) + '/' + flights['DAY'].astype(str) + '/' + flights['YEAR'].astype(str)
flights.drop(axis=1,inplace=True,labels=['MONTH','DAY','YEAR','CANCELLED','CANCELLATION_REASON','DAY_OF_WEEK','FLIGHT_NUMBER','DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME','AIR_TIME','DISTANCE','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [6]:
#clean missing values to their zero value to allow math operations to work
flights['AIR_SYSTEM_DELAY'].fillna(value=0,inplace=True)
flights['SECURITY_DELAY'].fillna(value=0,inplace=True)
flights['AIRLINE_DELAY'].fillna(value=0,inplace=True)
flights['LATE_AIRCRAFT_DELAY'].fillna(value=0,inplace=True)
flights['WEATHER_DELAY'].fillna(value=0,inplace=True)

In [7]:
# create two separate frames, one with inbound flights and one with outbound flights, and clean out un-needed data
ATLin = flights[flights['DESTINATION_AIRPORT'] == 'ATL']
ATLin.reset_index(inplace=True,drop=True)
ATLin = ATLin[ATLin['DIVERTED']==0]
ATLin.drop(axis=1,inplace=True,labels=['SCHEDULED_ARRIVAL','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','DIVERTED','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])

ATLout = flights[flights['ORIGIN_AIRPORT'] == 'ATL']
ATLout.reset_index(inplace=True,drop=True)
ATLout.drop(axis=1,inplace=True,labels=['ORIGIN_AIRPORT','SCHEDULED_ARRIVAL'])

C:\Users\frype\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### inbound to-do list
- determine total number of incoming flights for each day
- determine total cost to airport for air_system_delay for each day

In [8]:
#retrieve air system delay fee from airports information
ATL_ASD_FEE_RATE = airports[airports['IATA_CODE'] == 'ATL']['AIR SYSTEM DELAY FEE / MIN'].reset_index(drop=True)[0]

In [9]:
#air system delay costs for each flight
ATLin['AIR_SYSTEM_DELAY_COST'] = ATLin['AIR_SYSTEM_DELAY'] * ATL_ASD_FEE_RATE
ATLin['INBOUND FLIGHTS'] = 1

In [10]:
#merge all inbound information to the daily information data frame
byDate = pd.merge(byDate, ATLin.groupby('DATE').sum().reset_index(), on='DATE',how='outer')

In [11]:
byDate['AIR_SYSTEM_DELAY'].fillna(0)
byDate['AIR_SYSTEM_DELAY_COST'].fillna(0)

0        9593.7
1       74146.0
2      161791.1
3      125736.9
4       42223.6
         ...   
360     95257.8
361    147556.2
362     45025.3
363    499579.9
364      8886.2
Name: AIR_SYSTEM_DELAY_COST, Length: 365, dtype: float64

### outbount to-do list
For each day:
- determine total number of outbound flights
- calculate gate fees
- calculate airline-delay fee income
- calculate security delay fee

In [12]:
ATLout.drop(axis=1,labels=['AIRLINE','TAIL_NUMBER','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','DIVERTED','AIR_SYSTEM_DELAY'],inplace=True)

ATL_BGUC = airports[airports['IATA_CODE'] == 'ATL']['BASE GATE USE COST'].reset_index(drop=True)[0]
ATL_AD_FEE_RATE = airports[airports['IATA_CODE'] == 'ATL']['AIRLINE DELAY FEE / MIN'].reset_index(drop=True)[0]
ATL_SD_FEE_RATE = airports[airports['IATA_CODE'] == 'ATL']['SECURITY DELAY FEE / MIN'].reset_index(drop=True)[0]

ATLout['OUTBOUND FLIGHTS'] = 1
ATLout['GATE_REVENUE'] =  ATL_BGUC
ATLout['SECURITY_DELAY_COST'] = ATLout['SECURITY_DELAY'] * ATL_SD_FEE_RATE
ATLout['AIRLINE_DELAY_REVENUE'] = ATLout['AIRLINE_DELAY'] * ATL_AD_FEE_RATE

C:\Users\frype\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\frype\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\frype\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [13]:
#merge the grouped days into the date based frame with the information of inbound flights
byDate = pd.merge(byDate, ATLout.groupby('DATE').sum().reset_index(), on = 'DATE', how='outer')

In [14]:
#calculate overall daily net profits (revenues-costs)
byDate['PROFIT'] = byDate['GATE_REVENUE'] + byDate['AIRLINE_DELAY_REVENUE'] - byDate['SECURITY_DELAY_COST'] - byDate['AIR_SYSTEM_DELAY_COST']
byDate.to_csv('ATL_daily_profit.csv')
byDate #display output while the columns aren't in a logical order, we don't need them to be for visualization in tableau

,DATE,AIR_SYSTEM_DELAY,AIR_SYSTEM_DELAY_COST,INBOUND FLIGHTS,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OUTBOUND FLIGHTS,GATE_REVENUE,SECURITY_DELAY_COST,AIRLINE_DELAY_REVENUE,PROFIT
0,1/1/2015,339.0,9593.7,702.0,0.0,1285.0,543.0,0.0,738.0,3759512.22,0.00,54548.25,3804466.77
1,1/2/2015,2620.0,74146.0,1041.0,0.0,3727.0,2808.0,693.0,1038.0,5287769.22,0.00,158211.15,5371834.37
2,1/3/2015,5717.0,161791.1,934.0,53.0,6633.0,6708.0,995.0,938.0,4778350.22,2624.56,281570.85,4895505.41
3,1/4/2015,4443.0,125736.9,1022.0,0.0,9115.0,9567.0,615.0,1016.0,5175697.04,0.00,386931.75,5436891.89
4,1/5/2015,1492.0,42223.6,998.0,5.0,4752.0,3164.0,213.0,1001.0,5099284.19,247.60,201722.40,5258535.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,12/27/2015,3366.0,95257.8,1027.0,0.0,6365.0,7979.0,713.0,1031.0,5252109.89,0.00,270194.25,5427046.34
361,12/28/2015,5214.0,147556.2,1005.0,23.0,9637.0,9209.0,1746.0,1006.0,5124755.14,1138.96,409090.65,5385150.63
362,12/29/2015,1591.0,45025.3,1012.0,73.0,8051.0,6294.0,242.0,1012.0,5155320.28,3614.96,341764.95,5448444.97
363,12/30/2015,17653.0,499579.9,1002.0,0.0,13273.0,36615.0,11139.0,1016.0,5175697.04,0.00,563438.85,5239555.99
